In [1]:
!pip install --upgrade tensorflow-gpu
!pip install --upgrade keras

Requirement already up-to-date: tensorflow-gpu in /usr/local/lib/python3.6/dist-packages (2.0.0)
Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
!nvidia-smi


!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

%load_ext tensorboard


TensorFlow 2.x selected.
2.1.0-rc1
Found GPU at: /device:GPU:0
Fri Jan  3 18:18:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    37W / 250W |    265MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:             

In [3]:
"""
[W, S, A, D, WA, WD, SA, SD, NK]
"""
W = [1, 0, 0, 0, 0, 0, 0, 0, 0]
S = [0, 1, 0, 0, 0, 0, 0, 0, 0]
A = [0, 0, 1, 0, 0, 0, 0, 0, 0]
D = [0, 0, 0, 1, 0, 0, 0, 0, 0]
WA = [0, 0, 0, 0, 1, 0, 0, 0, 0]
WD = [0, 0, 0, 0, 0, 1, 0, 0, 0]
SA = [0, 0, 0, 0, 0, 0, 1, 0, 0]
SD = [0, 0, 0, 0, 0, 0, 0, 1, 0]
NK = [0, 0, 0, 0, 0, 0, 0, 0, 1]
HEIGHT = 120
WIDTH = 160
DIR = "TrainingData/SHUFFLED-240K-16%W"
EPOCHS = 40
MODEL_NAME = ''
SAVE_DIR = 'D:/AutomateDriving/TrainedModels/SHUFFLED-240K-16%W'

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.1.0-rc1


In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
 Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def create_model():
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=(WIDTH,HEIGHT,1), kernel_size=(11,11),\
    strides=(4,4), padding='valid'))
    model.add(Activation('relu'))
    # Pooling 
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(2,2), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(1,1), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Passing it to a dense layer
    model.add(Flatten())
    # 1st Dense Layer
    model.add(Dense(4096, input_shape=(224*224*3,)))
    model.add(Activation('relu'))
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 2nd Dense Layer
    model.add(Dense(4096))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Dense Layer
    model.add(Dense(1000))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Output Layer
    model.add(Dense(9))
    model.add(Activation('softmax'))

    model.summary()

    # (4) Compile 
    model.compile(loss='categorical_crossentropy', optimizer='adam',\
    metrics=['accuracy'])

    return model

Using TensorFlow backend.


In [5]:
model = create_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 38, 28, 96)        11712     
_________________________________________________________________
activation_1 (Activation)    (None, 38, 28, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 14, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 14, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 4, 256)         2973952   
_________________________________________________________________
activation_2 (Activation)    (None, 9, 4, 256)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 2, 256)        

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from zipfile import ZipFile 
import os
zip_name = "/content/drive/My Drive/Datasets/SHUFFLED-240K-16%W.zip"

In [9]:
all_epochs = []
for epoch in range(EPOCHS):
    for i in range(32):
        print("--------------------------------------------")

        # logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        # tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

        file_name = "CAR_160x120_{}.npy".format(i)

        with ZipFile(zip_name, 'r') as zipObj:
            zipObj.extract(file_name)

        print(epoch, file_name)
        data = np.load(file_name, allow_pickle=True)
 

        train_x = np.array([i[0] for i in data]).reshape(-1, WIDTH, HEIGHT, 1)
        train_y = np.array([i[1] for i in data])


        history = model.fit(train_x, train_y,
                  batch_size=256,
                  epochs=1,
                  verbose=0,
                  #callbacks=[tensorboard_callback],)
                  validation_split=0.1)
        
        
        print(history.history)
        #epoch_history.append(history)
        os.remove(file_name)

    model.save('model-SHUFFLED-240K-16%W-{}.h5'.format(epoch))
    all_epochs.append(history.history)


1 CAR_160x120_8.npy
{'val_loss': [1.8575192400626601], 'val_accuracy': [0.40740740299224854], 'loss': [1.158800827255479], 'accuracy': [0.58963805]}
--------------------------------------------
1 CAR_160x120_9.npy
{'val_loss': [1.0976113924671311], 'val_accuracy': [0.6224328875541687], 'loss': [0.6512019578547421], 'accuracy': [0.7574214]}
--------------------------------------------
1 CAR_160x120_10.npy
{'val_loss': [2.0165787942987783], 'val_accuracy': [0.38910505175590515], 'loss': [1.072853290154722], 'accuracy': [0.6627739]}
--------------------------------------------
1 CAR_160x120_11.npy
{'val_loss': [1.6635255610748327], 'val_accuracy': [0.40675991773605347], 'loss': [1.1610257329874054], 'accuracy': [0.61469674]}
--------------------------------------------
1 CAR_160x120_12.npy
{'val_loss': [1.5623265648655214], 'val_accuracy': [0.5911527872085571], 'loss': [0.9784895573663193], 'accuracy': [0.65663636]}
--------------------------------------------
1 CAR_160x120_13.npy
{'val_l

KeyboardInterrupt: ignored